In [1]:
# 1. Get 10,000 posts

In [3]:
import sys
import requests
calais_url = 'https://api.thomsonreuters.com/permid/calais'

import webhoseio, os
from gensim.models import KeyedVectors
import json
from scipy import spatial
from simhash import Simhash, SimhashIndex
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk, re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, sent_tokenize, ngrams
stopwords = set(nltk.corpus.stopwords.words('english'))

import spacy
import en_core_web_sm

In [2]:
webhoseio.config(token="2c45808b-852d-4156-af2f-faf960992ea9")
query_params = {"q": "organization:apple language:english published:>1533099600000", 
                "sort": "crawled"}
output = webhoseio.query("filterWebContent", query_params)
print(output['posts'][0]['text']) # Print the text of the first post
print(output['posts'][0]['published']) # Print the text of the first post publication date
# Print the site of the first post
print(output['posts'][0]['thread']['site'])

feeds = []
for item in output['posts']:
    feeds.append(item)
    #print(item['title'])
print(len(feeds))

count = 99
while count > 0:    
    output = webhoseio.get_next()
    for item in output['posts']:
        feeds.append(item)
    count -= 1
print(len(feeds)) #try org:google to get 10,000 feeds

with open("anly540_final.json", "w") as myfile: 
    for feed in feeds:
        line = json.dumps(feed)
        myfile.write(line)
        myfile.write("\n")

By Gun Talk Media in Gun Rights
The following video is brought to you courtesy of the Gun Talk Media YouTube Channel. Click the video below to watch it now.
Tom talks hunting ammo and more with Black Hills Ammunition’s Jeff Hoffman. Also, a range report on a 1911 CMP.
Tom Gresham’s Gun Talk® airs live on Sundays from 2PM-5PM Eastern, and runs on hundreds of radio stations nationwide. This gun-centric show features everything from new products to politics. Listen live on a radio station near you or via streaming. Listen to past shows now on Gun Talk’s channels on YouTube, Roku, Amazon Fire and Apple TV, and at guntalk.libsyn.com, Apple iTunes, i-Heart, Stitcher, the GunDealio app for iPhone or Android, and at guntalk.com.
©2018 Gun Talk Radio, LLC. All rights reserved. Recommended Viewing EXCLUSIVE: Claim Your Free 72-Hour Survival Food Kit Now…
In a crisis, your #1 need is food.
But not just any food...
Experts say everyone needs to have non-perishable, good for 25 years survival food 

In [19]:
# load json file
json_data = open("anly540_final.json").readlines()
feeds_read_from_file = []
for line in json_data:
    feeds_read_from_file.append(json.loads(line))
print(len(feeds_read_from_file))

8019


In [5]:
# 2. Deduplication

In [4]:
# Loading Word2Vec Google News model
model_path = './'
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec Google News', 'GoogleNews-vectors-negative300.bin', True)

Loading Word2Vec Google News model...
Finished loading Word2Vec Google News model...


In [7]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [8]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [9]:
def cleanup(input):
    # remove English stopwords
    input = input.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
    input = re.sub(r'[^a-zA-Z0-9 ]', '', input)
    return input

In [33]:
# Original set of Titles and Publish dates
feeds = []
for i, feed in enumerate(feeds_read_from_file):
    feed['title'] = cleanup(feed['title']).lower()
    feeds.append(feed)
#     print(feed['title'], '\n')
#     print(feed['text'], '\n')
#     print(feed['url'], '\n')
#print(feeds_read_from_file[0]['title'])

In [38]:
# Clean identical titles
dups = set()
for i in range(len(feeds)):
    for j in range(i + 1, len(feeds)):
        if j not in dups and feeds[i]['title'] == feeds[j]['title']:
            dups.add(j)
print(len(dups))

2245


In [39]:
# Get unique feeds for further deduplication process
feeds_unique = []
for i, feed in enumerate(feeds):
    if i not in dups:
        feeds_unique.append(feed)

In [40]:
for i, feed in enumerate(feeds_unique):
    feed['id'] = i
# feeds_unique
len(feeds_unique)

5774

In [41]:
# Set up similarity distance as 7 for further deduplication process
distance = 8
objs = [(str(feed['id']), Simhash(str(feed['title']))) for feed in feeds_unique]
index = SimhashIndex(objs, k=distance)
# index.bucket_size()

In [42]:
# indices of duplicated titles
dups = set()
for feed in feeds_unique:
    # Select a feed from the list based on its index
    SELECTED_INDEX = feed['id'] #27
    if str(SELECTED_INDEX) in dups:
        continue
    feed_sel = feeds_unique[SELECTED_INDEX]
    # calculate  hash value
    feed_hash = Simhash(str(feed_sel['title']))
    # find all duplicate indices
    dup_indices = index.get_near_dups(feed_hash)
    counts = 0
    for dup_index in dup_indices:
        if dup_index != str(SELECTED_INDEX):
            score = calc_similarity(feeds_unique[int(dup_index)]['title'], 
                                    feeds_unique[SELECTED_INDEX]['title'], 
                                    model_word2vec)
            
            if score > 0.8:
                dups.add(dup_index)
                counts += 1
                print("Dup#" + dup_index + ": score: " + str(score) + " " + feeds_unique[int(dup_index)]['title'])
#     print(dups)
    if counts >= 1: 
        print("Original#" + str(SELECTED_INDEX) + ": " + feed_sel['title'])
        print("Number of duplicates (SimHash + WordVectors): " + str(counts))
        print("")

Dup#181: score: 0.9785082016982468 apple is losing the interest of some bigmoney investors rbc says
Original#15: apple is losing the interest of some bigmoney investors rbc says aapl
Number of duplicates (SimHash): 1

Dup#229: score: 1.0 stocks struggle higher as markets remain volatile oil drops
Dup#413: score: 0.95286133414181 stocks struggle higher as markets remain volatile
Original#39: stocks struggle higher as markets remain volatile oil drops  kslcom
Number of duplicates (SimHash): 2

Dup#4362: score: 0.9300393773819554 global and chinese industrial internetofthings industry 2018 market research report
Dup#4372: score: 0.9008036496954177 global and chinese b2b publishing industry 2018 market research report
Original#40: global and chinese gaming industry 2018 market research report
Number of duplicates (SimHash): 2

Dup#356: score: 0.9760920048766812 china bans most iphone sales after granting qualcomm an injunction against apple
Dup#273: score: 0.9390729176365201 china bans mos

Dup#874: score: 0.9758314934249618 china bans the sale of iphone 6s 6s plus 7 7 plus 8 8 plus and x
Original#526: china bans the sale of iphone 6s 6s plus 7 7 plus 8 8 plus and x  phones
Number of duplicates (SimHash): 1

Dup#606: score: 0.9703371104595031 apple launches payments service in germany following delay  news  wtaq
Dup#2527: score: 0.9564871283694298 apple launches payments use in germany following delay
Dup#605: score: 0.9703371104595031 apple launches payments service in germany following delay  news  wtvb
Original#537: apple launches payments service in germany following delay
Number of duplicates (SimHash): 3

Dup#751: score: 0.9887091109647623 asian stocks gain apple suppliers outperform
Original#539: asian stocks gain apple suppliers outperform by investingcom
Number of duplicates (SimHash): 1

Dup#569: score: 0.9850009887246847 microsoft apple and others criticize australian governments new encryption law
Original#559: microsoft apple and others criticize australian g

Dup#5391: score: 1.0 apple denies iphone sales ban in china 
Dup#4378: score: 0.9678379048713261 apple denies iphone sales ban in china  phones
Original#975: apple denies iphone sales ban in china
Number of duplicates (SimHash): 2

Dup#1030: score: 0.9672692487382666 chinese companies call for us boycott amid huawei arrest backlash  cnn
Original#1023: chinese companies call for us boycott amid huawei arrest backlash
Number of duplicates (SimHash): 1

Dup#1139: score: 0.9949881774335565 smartphone industry to grow in 2019 after falling in 2018 says idc
Original#1031: smartphone industry to grow in 2019 after falling in 2018 says idc  marketwatch
Number of duplicates (SimHash): 1

Dup#1574: score: 0.9790923154049238 audit backs apples denial of chinese spy chips in servers
Original#1040: audit backs apple  denial of chinese spy chips in servers
Number of duplicates (SimHash): 1

Dup#3531: score: 0.9564831448173805 apple iphone xs max review
Original#1046: apple iphone xs max 390
Number o

Dup#2072: score: 0.9999999999999999 these are the latest countries  carriers to support esim on iphone xs  xr according to apple
Original#1838: these are the latest countries  carriers to support esim on iphone xs  xr according to apple  9to5mac
Number of duplicates (SimHash): 1

Dup#1869: score: 0.9290064664191685 chinese companies urge employees to boycott apple products  on air videos
Original#1844: chinese companies urge employees to boycott apple products
Number of duplicates (SimHash): 1

Dup#1861: score: 0.9999999999999998 update 4china ruling could ban some apple iphones sales amid qualcomm fight
Original#1860: update 5china ruling could ban some apple iphones sales amid qualcomm fight
Number of duplicates (SimHash): 1

Dup#2079: score: 0.9747920376947619 deep losses leave big tech with small earnings multiples
Original#1862: graphicdeep losses leave big tech with small earnings multiples
Number of duplicates (SimHash): 1

Dup#2011: score: 0.9915595646616859 google hearings for

Dup#3546: score: 0.9686918569727252 apple is considering moving iphone production out of china if tariffs hit 25
Original#2541: apple is considering moving iphone production out of china if tariffs hit 25  financial post
Number of duplicates (SimHash): 1

Dup#2857: score: 0.973759671752037 the annual end of year apple event at bh delivers some of 2018s best prices
Original#2545: new post the annual end of year apple event at bh delivers some of 2018s best prices
Number of duplicates (SimHash): 1

Dup#3320: score: 1.0 hemispherx biopharma issues letter to stockholders
Original#2574: hemispherx biopharma  issues letter to stockholders
Number of duplicates (SimHash): 1

Dup#2630: score: 1.0 apples netflix for magazines getting a chilly reception 
Dup#2618: score: 0.9313170337263232 apples netflix for magazines getting a chilly reception  news  analysis
Original#2579: apples netflix for magazines getting a chilly reception
Number of duplicates (SimHash): 2

Dup#2682: score: 1.0000000000000

Dup#3433: score: 0.9897966302432599 apple is quietly discounting the homepod  but there  a catch if you want the deal
Dup#3256: score: 0.9846190381002718 apple is quietly discounting the homepod  but theres a catch if you want the deal aapl
Original#3201: apple is quietly discounting the homepod  but there  a catch if you want the deal aapl
Number of duplicates (SimHash): 2

Dup#3221: score: 0.9620465890442338 recognizing the 21st century inventors iv
Original#3213: recognizing the 21st century inventors ii
Number of duplicates (SimHash): 1

Dup#3764: score: 0.9878417957602655 apple now has dozens of doctors on staff showing its serious about health tech
Original#3223: apple now has dozens of doctors on staff showing it  serious about health tech
Number of duplicates (SimHash): 1

Dup#3475: score: 0.9590143663614057 heres where you can still buy apples incredibly popular and hardtofind airpods this holiday season
Original#3238: here  where you can still buy apple  incredibly popular an

Number of duplicates (SimHash): 1

Dup#3649: score: 0.978356665198808 us tribunal to review ruling on qualcomm request for iphone ban  news  wibq
Original#3790: us tribunal to review ruling on qualcomm request for iphone ban  news  win 985
Number of duplicates (SimHash): 1

Dup#3649: score: 1.0 us tribunal to review ruling on qualcomm request for iphone ban  news  wibq
Original#3791: us tribunal to review ruling on qualcomm request for iphone ban  news  mixfm
Number of duplicates (SimHash): 1

Dup#3823: score: 0.9999999999999998 brand new 12 kids bike  fully assembled and tuned with warranty the bike truck  216 hoyt st 100
Original#3822: brand new 16 kids bike  fully assembled and tuned with warranty the bike truck  216 hoyt st 125
Number of duplicates (SimHash): 1

Dup#3878: score: 1.0000000000000002 techeia news update december 13 2018 at 1234pm
Original#3833: techeia news update december 13 2018 at 1141am
Number of duplicates (SimHash): 1

Dup#4344: score: 0.9694054546769872 vodafon

Dup#4342: score: 0.965779165061819 austin texas  apple announces plan to build 1 billion campus in texas
Original#4359: apple announces plan to build 1 billion campus in texas  koaacom
Number of duplicates (SimHash): 1

Dup#4434: score: 0.961963412008337 austin texas  apple deepens austin ties expands operations east and west
Dup#4492: score: 0.9999999999999999 apple deepens austin ties expands operations east and west  wwl
Original#4360: apple deepens austin ties expands operations east and west
Number of duplicates (SimHash): 2

Dup#4366: score: 0.9999999999999998 apple goes big in austin ecb meeting brexit nerves  ktvqcom
Original#4363: apple goes big in austin ecb meeting brexit nerves  kpaxcom
Number of duplicates (SimHash): 1

Dup#4368: score: 1.0000000000000002 apple announces plan to build 1 billion campus in texas  kvoacom
Original#4365: apple announces plan to build 1 billion campus in texas  ksbycom
Number of duplicates (SimHash): 1

Dup#4373: score: 0.9999999999999998 apple

Dup#5453: score: 0.9784468252503462 apple unveils plan for 1 bn campus in texas us expansion
Dup#4876: score: 0.9521660532422798 apple unveils plan for 1bn campus in texas us expansion
Original#4677: apple unveils plan for 1 bln campus in texas us expansion
Number of duplicates (SimHash): 2

Dup#3859: score: 0.949822793559682 japan plans tighter regulation of tech giants east
Original#4691: japan plans tighter regulation of tech giants east asia
Number of duplicates (SimHash): 1

Dup#5185: score: 0.837465236605673 apple deepens austin ties expands operations east and west  thu 13 dec 2018 pst
Original#4703: apple deepens austin ties expands us operations east and west
Number of duplicates (SimHash): 1

Dup#4906: score: 0.9789523099779356 apple announces plan to build new 1 billion campus in austin
Original#4715: apple announces plan to build 1 billion austin campus
Number of duplicates (SimHash): 1

Dup#4800: score: 0.9017056378742093 update 4apple deepens ties with austin to spend 1 b

Dup#5234: score: 1.0000000000000002 apple  plans billiondollar austin campus in wave of new sites  update
Dup#5297: score: 0.9591737866326003 apple plans billiondollar austin campus in wave of new sites
Original#5204: apple plans billiondollar austin campus in wave of new sites  update
Number of duplicates (SimHash): 2

Dup#5241: score: 0.9172491905904162 apple could be planning to cut out another iphone supplier
Original#5237: apple could be planning to cut out another iphone supplier  the motley fool
Number of duplicates (SimHash): 1

Dup#5397: score: 0.9452228381948177 apple is not just building a new campus in austin heres everywhere the tech giant is expanding  business insider  
Dup#5383: score: 0.970808314962637 apple is not just building a new campus in austin heres everywhere the tech giant is expanding
Dup#5275: score: 0.9914802498159208 apple isnt just building a new campus in austin here  everywhere the tech giant is expanding aapl
Original#5259: apple isnt just building a 

In [44]:
# Clean up deduplicated dataset
feeds_unique_deduped = []
for i, feed in enumerate(feeds_unique):
    if str(feed['id']) not in dups:
        feeds_unique_deduped.append(feed)
len(feeds_unique_deduped)
len(dups)

370

In [45]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is np.nan:
        result = 0
        
    return result

In [46]:
# Save as txt file and print deduplicated subset of id, titles and published dates
# dup_indices_set = set(dup_indices)
with open('./deduped_apple.txt', 'w') as f:
    for feed in feeds_unique_deduped:
#         print(' '.join((str(feed['id']), str(feed['title']),  feed['published'])))
        f.write(' '.join((str(feed['id']), str(feed['title']),  feed['published'], '\n')))

In [47]:
#3. Named Entity Recognition

In [48]:
def sendFile(input_data, headers, output_dir='./'):
#     with open(file_name, 'rb') as input_data:
    response = requests.post(calais_url, data=input_data, headers=headers, timeout=80)
    print ('status code: %s' % response.status_code)
    content = response.text
#     print ('Results received: %s' % content)
    if response.status_code == 200:
        saveFile('final_text_apple', output_dir, content)
        return response
            
def saveFile(file_name, output_dir, content):
    output_file_name = os.path.basename(file_name) + '.json'
    output_file = open(os.path.join(output_dir, output_file_name), 'wb')
    output_file.write(content.encode('utf-8'))
    output_file.close()

In [50]:
def extract_entity(tagging_result):
    for hash in tagging_result:        
        if '_typeGroup' in tagging_result[hash] and tagging_result[hash]['_typeGroup'] == 'entities':
            print(tagging_result[hash]['_type'], ':', 
                  tagging_result[hash]['name'], ', relevance: ', 
                  tagging_result[hash]['relevance'])

Technology : android , relevance:  0.8


In [49]:
RANDOM_INDEX = 7
print(feeds_unique_deduped[RANDOM_INDEX]['title'])
access_token = 'ZR2BvCL0pxKthAeKa0FKPbb9I9At9ows'
headers = {'X-AG-Access-Token' : access_token, 
           'Content-Type' : 'text/raw', 
           'outputformat' : 'application/json'
          }

for feed in feeds_unique_deduped:
    input_data = feed['title'].encode('utf-8')
    response = sendFile(input_data, headers)
    tagging_result = json.loads(response.content)
    extract_entity(tagging_result)

apples w1 chip what is it and why do not android users get a bite
status code: 200


In [231]:
#4. Topic Clusters

In [91]:
feed_titles = []
for feed in feeds_unique_deduped:
#         print(feed['title'])
        feed_titles.append(str(feed['title']))
print("Total number of titles: " + str(len(feed_titles)))

Total number of titles: 5404


['hunting ammo 1911 cmp range report gun talk radio  12918 c',
 'swapping slides personal defense loads supporting the nra gun talk radio  12918 after show',
 'blue book of gun values  new ar15 guide gun talk radio  12918 a',
 'hey siri when is my next flight  andrew',
 'apple watch  delta airlines  complications  andrew',
 'apples iphone xs max beats huaweis mate 20 pro in speed test',
 'panasonic ergofit earbuds  lowest price',
 'apples w1 chip what is it and why do not android users get a bite',
 'the lsat is going digital exclusively on microsoft surface go tablets  the verge',
 'apple launches a new site to promote tradein towards iphone xs and iphone xr',
 'westleaf cannabis is redefining retail across canada',
 'apple appeals iphone ruling in china',
 'ipad 2 3g apple 80',
 'apple  hit with iphone sales ban in china qualcomm says  4th update  marketscreener',
 'trello acquires butler to add power of automation',
 'apple is losing the interest of some bigmoney investors rbc says 

In [58]:
def tokenize_titles(title):
    tokens = nltk.word_tokenize(title)
    lmtzr = WordNetLemmatizer()
    filtered_tokens = []
    
    for token in tokens:
        token = token.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
        token = re.sub(r'[^a-zA-Z0-9 ]', '', token)
        if token not in stopwords:
            filtered_tokens.append(token.lower())
    
    lemmas = [lmtzr.lemmatize(t,'v') for t in filtered_tokens]

    return lemmas

In [92]:
def clstr_lda(num_topics, titles):
    # top words to be identified
    n_top_words = 10

    tf_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000,
                                    tokenizer=tokenize_titles, ngram_range=(3,4))

    tf = tf_vectorizer.fit_transform(titles)

    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=1000,
                                    learning_method='batch', learning_offset=10.,
                                    random_state = 1)
    lda.fit(tf)
    tf_feature_names = tf_vectorizer.get_feature_names()

    # print top topic words
    topics = dict()
    for topic_idx, topic in enumerate(lda.components_):
        topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print("Topic #%d:" % topic_idx)
        print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
    return topics

In [95]:
topics = clstr_lda(10, feed_titles)

Topic #0:
apple build new | apple iphone 8 | new campus austin | build new campus | apple build new campus | build new campus austin | iphone 8 plus | job across us | apple iphone 8 plus | china sales ban
Topic #1:
apple inc aapl | new austin campus | invest 1 billion | samsung galaxy s10 | apple invest 1 | apple watch 3 | apple invest 1 billion | 1 billion austin | apple announce new | iphone 6s 64gb
Topic #2:
xs xs max | iphone xs xs | iphone xs xs max | apple watch series | watch series 1 | apple watch series 1 | iphone sales ban | austin 1 billion | xs max xr | 1 billion campus
Topic #3:
apple watch series | watch series 4 | apple watch series 4 | iphone 7 plus | apple music subscribers | watch series 3 gps | series 3 gps | iphone xr xs | apple iphone 7 plus | apple iphone x
Topic #4:
ban iphone sales | china ban iphone | apple iphone xr | chinese court ban | apple iphone 6s | china court ban | chinese spy chip | apple macbook air | court ban iphone | apple ipad pro
Topic #5:
new 1

In [ ]:
#5. Topic Taxonomy

In [252]:
nlp = en_core_web_sm.load()
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
OBJECTS = ["dobj", "dative", "attr", "oprd"] 

In [274]:
for feed in feeds_unique_deduped[:1]:
    selected_feed = feed['text']
#     print(selected_feed) 
    sentences = sent_tokenize(selected_feed)
    for sentence in sentences:
        print(sentence)
        print()

So is this it — the end of the decade-long bull market?

Or is it the sort of normal, healthy pullback that often follows a strong showing?

Last week, it felt like the bottom dropped out of the stock market.

The blue-chip Dow Jones industrial average fell 4.5 percent for the week, ending with a 559-point rout Friday that left the index in negative territory for the year.

The benchmark Standard & Poor's 500-stock index has fallen into a "correction," meaning it's off more than 10 percent from its peak reached Sept. 20.

And once-highflying tech stocks faltered with Apple, Facebook and Google parent Alphabet all ending last week showing stock-price losses for the year — driving the Nasdaq composite index into a correction too.

So is this it — the end of the decade-long bull market?

Or is it the sort of normal, healthy pullback that often follows a strong showing?

"Everyone is trying to figure that out," said Sam Stovall, chief investment strategist at CFRA Research, though he's amo

In [275]:
def getSubsFromConjunctions(subs):
    moreSubs = []
    for sub in subs:
        # rights is a generator
        rights = list(sub.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreSubs.extend([tok for tok in rights if tok.dep_ in SUBJECTS or tok.pos_ == "NOUN"])
            if len(moreSubs) > 0:
                moreSubs.extend(getSubsFromConjunctions(moreSubs))
    return moreSubs

In [276]:
def getObjsFromConjunctions(objs):
    moreObjs = []
    for obj in objs:
        # rights is a generator
        rights = list(obj.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreObjs.extend([tok for tok in rights if tok.dep_ in OBJECTS or tok.pos_ == "NOUN"])
            if len(moreObjs) > 0:
                moreObjs.extend(getObjsFromConjunctions(moreObjs))
    return moreObjs

In [277]:
def getVerbsFromConjunctions(verbs):
    moreVerbs = []
    for verb in verbs:
        rightDeps = {tok.lower_ for tok in verb.rights}
        if "and" in rightDeps:
            moreVerbs.extend([tok for tok in verb.rights if tok.pos_ == "VERB"])
            if len(moreVerbs) > 0:
                moreVerbs.extend(getVerbsFromConjunctions(moreVerbs))
    return moreVerbs

In [278]:
def findSubs(tok):
    head = tok.head
    while head.pos_ != "VERB" and head.pos_ != "NOUN" and head.head != head:
        head = head.head
    if head.pos_ == "VERB":
        subs = [tok for tok in head.lefts if tok.dep_ == "SUB"]
        if len(subs) > 0:
            verbNegated = isNegated(head)
            subs.extend(getSubsFromConjunctions(subs))
            return subs, verbNegated
        elif head.head != head:
            return findSubs(head)
    elif head.pos_ == "NOUN":
        return [head], isNegated(tok)
    return [], False

In [279]:
def isNegated(tok):
    negations = {"no", "not", "n't", "never", "none"}
    for dep in list(tok.lefts) + list(tok.rights):
        if dep.lower_ in negations:
            return True
    return False

In [280]:
def findSVs(tokens):
    svs = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        if len(subs) > 0:
            for sub in subs:
                svs.append((sub.orth_, "!" + v.orth_ if verbNegated else v.orth_))
    return svs

In [281]:
def getObjsFromPrepositions(deps):
    objs = []
    for dep in deps:
        if dep.pos_ == "ADP" and dep.dep_ == "prep":
            objs.extend([tok for tok in dep.rights if tok.dep_  in OBJECTS or (tok.pos_ == "PRON" and tok.lower_ == "me")])
    return objs

In [282]:
def getObjsFromAttrs(deps):
    for dep in deps:
        if dep.pos_ == "NOUN" and dep.dep_ == "attr":
            verbs = [tok for tok in dep.rights if tok.pos_ == "VERB"]
            if len(verbs) > 0:
                for v in verbs:
                    rights = list(v.rights)
                    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
                    objs.extend(getObjsFromPrepositions(rights))
                    if len(objs) > 0:
                        return v, objs
    return None, None

In [283]:
def getObjFromXComp(deps):
    for dep in deps:
        if dep.pos_ == "VERB" and dep.dep_ == "xcomp":
            v = dep
            rights = list(v.rights)
            objs = [tok for tok in rights if tok.dep_ in OBJECTS]
            objs.extend(getObjsFromPrepositions(rights))
            if len(objs) > 0:
                return v, objs
    return None, None

In [284]:
def getAllSubs(v):
    verbNegated = isNegated(v)
    subs = [tok for tok in v.lefts if tok.dep_ in SUBJECTS and tok.pos_ != "DET"]
    if len(subs) > 0:
        subs.extend(getSubsFromConjunctions(subs))
    else:
        foundSubs, verbNegated = findSubs(v)
        subs.extend(foundSubs)
    return subs, verbNegated

In [285]:
def getAllObjs(v):
    # rights is a generator
    rights = list(v.rights)
    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
    objs.extend(getObjsFromPrepositions(rights))

    #potentialNewVerb, potentialNewObjs = getObjsFromAttrs(rights)
    #if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
    #    objs.extend(potentialNewObjs)
    #    v = potentialNewVerb

    potentialNewVerb, potentialNewObjs = getObjFromXComp(rights)
    if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
        objs.extend(potentialNewObjs)
        v = potentialNewVerb
    if len(objs) > 0:
        objs.extend(getObjsFromConjunctions(objs))
    return v, objs

In [286]:
def findSVOs(tokens):
    svos = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        # hopefully there are subs, if not, don't examine this verb any longer
        if len(subs) > 0:
            v, objs = getAllObjs(v)
            for sub in subs:
                for obj in objs:
                    objNegated = isNegated(obj)
                    svos.append((sub.lower_, "!" + v.lower_ if verbNegated or objNegated else v.lower_, obj.lower_))
    return svos

In [287]:
def getAbuserOntoVictimSVOs(tokens):
    maleAbuser = {'he', 'boyfriend', 'bf', 'father', 'dad', 'husband', 'brother', 'man'}
    femaleAbuser = {'she', 'girlfriend', 'gf', 'mother', 'mom', 'wife', 'sister', 'woman'}
    neutralAbuser = {'pastor', 'abuser', 'offender', 'ex', 'x', 'lover', 'church', 'they'}
    victim = {'me', 'sister', 'brother', 'child', 'kid', 'baby', 'friend', 'her', 'him', 'man', 'woman'}

    svos = findSVOs(tokens)
    wnl = WordNetLemmatizer()
    passed = []
    for s, v, o in svos:
        s = wnl.lemmatize(s)
        v = "!" + wnl.lemmatize(v[1:], 'v') if v[0] == "!" else wnl.lemmatize(v, 'v')
        o = "!" + wnl.lemmatize(o[1:]) if o[0] == "!" else wnl.lemmatize(o)
        if s in maleAbuser.union(femaleAbuser).union(neutralAbuser) and o in victim:
            passed.append((s, v, o))
    return passed

In [288]:
def printDeps(toks):
    for tok in toks:
        print(tok.orth_, tok.dep_, tok.pos_, tok.head.orth_, [t.orth_ for t in tok.lefts], [t.orth_ for t in tok.rights])

def testSVOs(sentences):
    #nlp = English()
    for sentence in sentences:
        tok = nlp(sentence)
        svos = findSVOs(tok)
    #     printDeps(tok)
    #    assert set(svos) == {('i', '!have', 'assistance'), ('he', '!provide', 'support')}
        if svos:
            for svo in svos:
                print('S:{0}, V:{1}, O:{2}'.format((svo[0]), svo[1], svo[2]))
        else:
            print('No SVO')

In [289]:
testSVOs()

No SVO
S:that, V:follows, O:showing
No SVO
S:that, V:left, O:index
S:it, V:'s, O:percent
S:stocks, V:faltered, O:all
No SVO
S:that, V:follows, O:showing
S:everyone, V:figure, O:that
S:obscuring, V:are, O:trends
S:that, V:keeping, O:pressure
S:trends, V:sparking, O:trading
S:they, V:include, O:tariffs
S:they, V:include, O:battle
S:they, V:include, O:prices
S:they, V:include, O:bursts
S:prices, V:raising, O:rates
S:pullback, V:is, O:—
No SVO
No SVO
No SVO
S:it, V:was, O:loss
S:market, V:reached, O:levels
S:s&p, V:gained, O:percent
No SVO
No SVO
S:we, V:had, O:amount
S:year, V:!was, O:anomaly
No SVO
No SVO
S:s&p, V:lost, O:percent
S:s&p, V:lost, O:decline
No SVO
S:yields, V:locking, O:money
No SVO
No SVO
S:investors, V:pay, O:attention
No SVO
S:it, V:be, O:buy
S:s&p, V:trading, O:earnings
No SVO
No SVO
S:street, V:expected, O:growth
S:we, V:see, O:amount
No SVO


In [ ]:
# Develop own taxonomy

In [125]:
topic_taxonomy = {
    "Business":
    {
        "Trading and Investing": "nasdaq billion stock share trade sale dow market invest acquire merge purchase subsidiary",
        "Company Corporation and Competition": "file suit compete partnership rivalry win lose deal",
        "Policy Change": "operation change organizational policy support transfer plan",
        "Operational Bottleneck": "order injunction appeal ban"
    },
    "Technology":
    {
        "Laptop": "microsoft mac pro air surface lenovo dell alienware asus apple",
        "Smart Phone": "iphone android nexus huawei xiaomi pixel xs max xr",
        "Intelligent Device": "alexa google home mini ipod ipad amazon watch series"
    }
} 

In [100]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is 'nan':
        result = 0
        
    return result

In [101]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [118]:
# function takes an input string, runs similarity for each item in topic_taxonomy, sorts and returns top 3 results
def classify_topics(input, vectors, topic_taxonomy):
    feed_score = dict()
    for key, value in topic_taxonomy.items():
        max_value_score = dict()
        for label, keywords in value.items():
            max_value_score[label] = 0
            topic = (key + ' ' + keywords).strip()
            max_value_score[label] += float(calc_similarity(input, topic, vectors))
            
#         print(max_value_score.items(), feed_score.items())    
        sorted_max_score = sorted(max_value_score.items(), key=lambda x : x[1], reverse=True)[0]
        feed_score[sorted_max_score[0]] = sorted_max_score[1]
    return sorted(feed_score.items(), key=lambda x : x[1], reverse=True)[:3]

In [128]:
# example using Word2Vec
output1 = classify_topics('apple is appealing qualcomms attempted ban of iphone sales in china', model_word2vec, topic_taxonomy)
print(output1)
output2 = classify_topics('dow sp dogged by growth worries nasdaq bounces', model_word2vec, topic_taxonomy)
print(output2)
output3 = classify_topics('with the series 4 apple mastered the smartwatch finally', model_word2vec, topic_taxonomy)
print(output3)
# # example output using FastText, note for FastText, string needs to be non-capitalized
# output2 = classify_topics('White House Has Plan To Force Tillerson Out Replace With CIA Chief'.lower(), model_fasttext)
# print(output2)
    

[('Intelligent Device', 0.527089404945571), ('Operational Bottleneck', 0.4609588926910678)]
[('Trading and Investing', 0.4629720958443735), ('Smart Phone', 0.36071347790552166)]
[('Intelligent Device', 0.3993975731220163), ('Company Corporation and Competition', 0.3067364645233541)]


In [127]:
for title in feed_titles:
    if "ban" in title:
        print(title)

apple  hit with iphone sales ban in china qualcomm says  4th update  marketscreener
qualcomm wins import ban against apple iphones models in china
chinese court bans some iphones over qualcomm dispute
lgbt community outraged after tumblr bans adult content  activist mommy
china grants iphone import and sale ban over qualcomm dispute but apple seemingly has a workaround u
trade war counter attack china slams apple with iphone sale ban over patent violations
china bans sale import of most iphones in qualcomm fight
china bans most iphone sales after granting qualcomm an injunction against apple  cnn
qualcomm says it won case banning sale of older iphones in china  the indian express
iphone ban in china may push apple qualcomm toward settlement 
apple hit with sales ban on older iphones in china
apple says no china ban on iphone sales
apple denies iphone import ban in china
apple is appealing qualcomms attempted ban of iphone sales in china
china to ban nearly all iphones for violating pat